In [379]:
import torch
import numpy as np

x = torch.from_numpy(np.random.randn(5)).requires_grad_(True)
y = torch.ones_like(x)
print(x, y)

z = x + y
print(z)

out = z.mean()
print(f"\n\n{out}\n")

out.backward()
print(x.grad)

tensor([ 0.5620,  0.5979,  1.4958,  0.1142, -1.6214], dtype=torch.float64,
       requires_grad=True) tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
tensor([ 1.5620,  1.5979,  2.4958,  1.1142, -0.6214], dtype=torch.float64,
       grad_fn=<AddBackward0>)


1.2296897851380213

tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], dtype=torch.float64)


In [381]:
#backward()的累加效应
out2 = x.sum()
print(out2)
out2.backward()
print(x.grad)

tensor(1.1484, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor([1.2000, 1.2000, 1.2000, 1.2000, 1.2000], dtype=torch.float64)


In [383]:
#清除x的梯度
x.grad.data.zero_()
out3 = x.sum()
out3.backward()
print(x.grad)

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [385]:
#x = torch.tensor([1, 2, 3, 4], requires_grad = True)  #会报错，整数不能求梯度
x = torch.tensor([1., 2., 3., 4.], requires_grad = True)
a = torch.tensor([2., 2., 2., 2.], requires_grad = True)
y = a * x
print(y)
z = y.view(2, 2)
print(z)
#z.backward() #z不是标量，直接backward()会报错
#需要传入一个与z同形状的v进行相乘求和成一个标量s才可以对s反向传播，公式如下

tensor([2., 4., 6., 8.], grad_fn=<MulBackward0>)
tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward0>)


$$
S = \langle a, b \rangle = \sum z_{ij}v_{ij}
$$

| 显示效果                         | LaTeX 代码                     |
| ---------------------------- | ---------------------------- |
| $(a+b)$                      | `(a+b)`                      |
| $[a+b]$                      | `[a+b]`                      |
| $\{a+b\}$                    | `\{a+b\}`                    |
| $\langle a, b \rangle$       | `\langle a, b \rangle`       |
| $\vert x \vert$              | `\vert x \vert`              |
| $\|x\|$                      | `\|x\|`                      |
| $\lfloor x \rfloor$          | `\lfloor x \rfloor`          |
| $\lceil x \rceil$            | `\lceil x \rceil`            |
| $\langle z, v \rangle$       | `\langle z, v \rangle`       |
| $\big( \frac{a}{b} \big)$    | `\big( \frac{a}{b} \big)`    |
| $\Big[ \frac{a}{b} \Big]$    | `\Big[ \frac{a}{b} \Big]`    |
| $\left( \frac{a}{b} \right)$ | `\left( \frac{a}{b} \right)` |


In [387]:
#对 z 或上游某个变量执行过 backward，图已经释放了，中间变量的值已经被释放了
#要么保留中间变量，要么重新正向传播
v = torch.tensor([[3, 0.1], [0.1311, 31]])
#retain_graph = True可以保留计算图中变量的值
z.backward(v, retain_graph = True)
#z.backward(v, retain_graph = True)不会改变z本身的值
print(z)
print(a.grad)
a.grad.data.zero_()

#a.grad和a是同形的张量

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward0>)
tensor([  3.0000,   0.2000,   0.3933, 124.0000])


tensor([0., 0., 0., 0.])

$$
\nabla_{x}s= 2\cdot\mathrm{flatten}(v)
$$

In [391]:
"""
with的简单理解

with expression [as variable]:

    do_something()

with 语句特点是：
1.expression必须返回一个 上下文管理器（context manager）对象
2.as variable：可选，用来接收 __enter__() 方法返回的对象。
3.代码块执行完后会自动调用 __exit__() 方法做清理工作。
4.不论是否运行出错都会结束运行
5.化简
    try:
    finally:
   语法 
"""

'\nwith的简单理解\n\nwith expression [as variable]:\n\n    do_something()\n\nwith 语句特点是：\n1.expression必须返回一个 上下文管理器（context manager）对象\n2.as variable：可选，用来接收 __enter__() 方法返回的对象。\n3.代码块执行完后会自动调用 __exit__() 方法做清理工作。\n4.不论是否运行出错都会结束运行\n5.化简\n    try:\n    finally:\n   语法 \n'

In [399]:
f = open('test.txt')
try:
    pass
finally:
    f.close()

In [401]:
#这段代码和前后没有关系请忽略，只是一个测试
import os
import platform

file_path = "test.txt"

if platform.system() == "Windows":
    os.startfile(file_path)        

In [403]:
#可以化简为如下
with open('test.txt') as f:  
    pass

In [419]:
#一个简单的例子说明with如何工作
class MyOpen:#定义一个上下文管理器的类
    def __init__(self, filepath):
        print('Entering construtor of MyOpen') #验证进入了初始化
        self.filepath = filepath
    def __enter__(self):
        print('Entering __enter__ of MyOpen') #验证进入了初始化
        return self.filepath #就是把return的这个self.filepath给了f
    def __exit__(self, exc_type, exc_value, traceback):#分别表示异常类型、异常实例、traceback，若没有异常，则都是None
        print('Entering the __exit__ of MyOpen')
        if exc_type is ValueError: #if语句后面记得接冒号  
            print('Caught a ValueError')
        return True

with MyOpen('test.txt') as f:
    raise ValueError('A ValueError occured') #加入异常仍能退出
    print(f'The value of f is {f}')

Entering construtor of MyOpen
Entering __enter__ of MyOpen
Entering the __exit__ of MyOpen
Caught a ValueError


In [758]:
x = torch.tensor(1., requires_grad = True)
#x的n次方
n = torch.tensor(2., requires_grad = True)
m = torch.tensor(3., requires_grad = True)
y1 = x ** n

with torch.no_grad():
    y2 = x ** m
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)#输出False，torch.no_grad()中断了requires_grad = True

y3 = y1 + y2
print(y3, y3.requires_grad)#True + False 还是True

tensor(1., grad_fn=<PowBackward1>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [774]:
#保存梯度
y1.retain_grad()
y3.retain_grad()

y3.backward(retain_graph = True)

print(x.grad)
x.grad.data.zero_()
print(n.grad)
n.grad.data.zero_()

#对y1和y3直接输出.grad会报错，因为要节省内存会直接释放非子叶节点的梯度，可以选择手动保存
print(x.is_leaf, n.is_leaf, y1.is_leaf, y3.is_leaf)
print(y1.grad)
y1.grad.data.zero_()
print(y3.grad)
y3.grad.data.zero_()

#下面两个会报错，因为y2.requires_grad = False
'''
print(y2.grad)
y2.grad.data.zero_()
print(m.grad)
m.grad.data.zero_()
'''

tensor(2.)
tensor(0.)
True True False False
tensor(1.)
tensor(1.)


'\nprint(y2.grad)\ny2.grad.data.zero_()\nprint(m.grad)\nm.grad.data.zero_()\n'

$$
y_{3}=y_{1}+y_{2} = x^{5}
$$

但是x.grad却是$2x$,因为y_{2} = x^{3}的requires_grad是False，所以梯度不会回传，所以y2.backward()会报错

### *注意*：
1. 一次正向传播无法重复.backward(),因为会释放计算图节省空间，解决方法：
<pre>    .backward(retain_graph=True)</pre>

2. .backward()不会保留非叶子张量的梯度，因为节省空间，解决方法：
<pre>    .retain_grad()
    .backward()</pre>

3. 梯度会累积，解决方法：
<pre>    .backward()
    .grad.data.zero_()

In [ ]:
"""
要修改值建议用torch.no_grad()，不要用.data
"""